In [1]:
import ROOT
import pandas as pd

Welcome to JupyROOT 6.26/10


In [2]:
df = pd.read_csv('test.csv', index_col=0)

In [3]:
df.columns

Index(['BeamSpotZ_x'], dtype='object')

In [5]:
beamspots = df.BeamSpotZ_x.to_numpy()
beamspot = ROOT.RooRealVar("Beamspot", "Beamspot", -5, 5, "cm")

In [10]:
data = ROOT.RooDataSet.from_numpy({"Beamspot": beamspots}, [beamspot])

[#0] WARNING:DataHandling -- RooDataSet.from_numpy() Ignored 1459 out-of-range events


In [19]:
x, s = getFittedValues(df)

[#0] WARNING:DataHandling -- RooDataSet.from_numpy() Ignored 1459 out-of-range events
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
 **********
 **   21 **SET PRINT           1
 **********
 **********
 **   22 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 mean         0.00000e+00  1.00000e+00   -5.00000e+00  5.00000e+00
     2 sigma        1.00000e-01  4.50000e-02    1.00000e-02  5.00000e+00
 **********
 **   23 **SET ERR         0.5
 **********
 **********
 **   24 **SET PRINT           1
 **********
 **********
 **   25 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **   26 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
RooAbsMinimizerFcn: Minimized function has error status.
Returning maximum FCN so far (-inf) to force MIGRAD to back out of this r

In [20]:
x, s

(-2.397127693021015, 0.10000000000000009)

In [18]:
def getFittedValues(df):
    beamspots = df.BeamSpotZ_x.to_numpy()
    beamspot = ROOT.RooRealVar("Beamspot", "Beamspot", -5, 5, "cm")
    data = ROOT.RooDataSet.from_numpy({f"Beamspot": beamspots}, [beamspot])

    # mean, std = fitTripleGauss(beamspot, data)
    mean, std = fitBinnedGauss(beamspot, data)
    return mean, std

In [15]:
def fitBinnedGauss(beamspot: ROOT.RooRealVar, data: ROOT.RooDataSet):
    beamspot.setBins(100)
    data_hist = ROOT.RooDataHist("data_hist", "data_hist", ROOT.RooArgList(beamspot), data)

    mean = ROOT.RooRealVar("mean", "mean", 0, -5, 5)
    sigma = ROOT.RooRealVar("sigma", "Sigma", 0.1, 0.01, 5)
    gauss = ROOT.RooGaussian("gauss", "gauss", beamspot, mean, sigma)

    gauss.fitTo(data_hist, ROOT.RooFit.Save())

    frame = beamspot.frame()
    data_hist.plotOn(frame)
    gauss.plotOn(frame)

    canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
    frame.Draw()
    canvas.Draw()
    frame.Draw()

    return mean.getVal(), sigma.getVal()

In [ ]:
def fitTripleGauss(beamspot: ROOT.RooRealVar, data: ROOT.RooDataSet):
    mean = ROOT.RooRealVar("mean", "mean", 0, -5, 5)
    sigma1 = ROOT.RooRealVar("sigma1", "Sigma1", 2, 0.1, 10)
    sigma2 = ROOT.RooRealVar("sigma2", "Sigma2", 2, 0.1, 10)
    sigma3 = ROOT.RooRealVar("sigma3", "Sigma3", 2, 0.1, 10)

    gauss1 = ROOT.RooGaussian("gauss1", "Gauss 1", beamspot, mean, sigma1)
    gauss2 = ROOT.RooGaussian("gauss2", "Gauss 2", beamspot, mean, sigma2)
    gauss3 = ROOT.RooGaussian("gauss3", "Gauss 3", beamspot, mean, sigma3)

    frac1 = ROOT.RooRealVar("frac1", "Fraction 1", 0.5, 0, 1)
    frac2 = ROOT.RooRealVar("frac2", "Fraction 2", 0.5, 0, 1)

    model = ROOT.RooAddPdf("model", "Model", ROOT.RooArgList(gauss1, gauss2, gauss3), ROOT.RooArgList(frac1, frac2))

    model.fitTo(data, ROOT.RooFit.Save())


    frame = beamspot.frame()
    data.plotOn(frame, ROOT.RooFit.Name('dataset'))
    model.plotOn(frame, ROOT.RooFit.Name('model'))
    model.plotOn(frame, ROOT.RooFit.Components("gauss1"), ROOT.RooFit.LineStyle(ROOT.kDashed))
    model.plotOn(frame, ROOT.RooFit.Components("gauss2"), ROOT.RooFit.LineStyle(ROOT.kDashed), ROOT.RooFit.LineColor(ROOT.kRed))
    model.plotOn(frame, ROOT.RooFit.Components("gauss3"), ROOT.RooFit.LineStyle(ROOT.kDashed), ROOT.RooFit.LineColor(ROOT.kGreen))

    chi2 = frame.chiSquare("model", "dataset", 6)

    model.paramOn(frame, ROOT.RooFit.Layout(0.6, 0.9, 0.9))
    frame.getAttText().SetTextSize(0.03)
    pt = frame.findObject("model_paramBox")
    pt.AddText(ROOT.Form(f"Chi2/ndof =  {chi2:.2f}"))

    # Canvas
    canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
    frame.Draw()
    canvas.Draw()

    return mean.getVal(), sigma1.getVal()